In [1]:
TF_ENABLE_ONEDNN_OPTS=0

In [2]:
import os 
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
from tqdm.auto import tqdm
import cv2

# from tensorflow.keras.applications import Densnet
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers
import tensorflow.keras as keras
from tensorflow.data import Dataset

from tensorflow.keras.optimizers import Adam, RMSprop, Nadam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

from itertools import product
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.applications import *
from sklearn.preprocessing import LabelEncoder

/home/ubuntu/anaconda3/envs/smartpot/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-06-09 13:59:23.369085: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


<img src='https://miro.medium.com/max/1300/1*J_r7zr0LOUfjgyvV6gNjbA.png'/>

In [3]:
class CustomDataGen(tf.keras.utils.Sequence):
    def __init__(self, df, X_col, y_col,
                 batch_size,
                 input_size=(224, 224, 3),
                 shuffle=True):
        
        self.df = df.copy()
        self.X_col = X_col
        self.y_col = y_col
        self.batch_size = batch_size
        self.input_size = input_size
        self.shuffle = shuffle
        
        self.n = len(self.df)
        self.n_cls = df[self.y_col['cls1']].nunique()

        self.cls1_en = LabelEncoder()
        self.cls1_en.fit(self.df[self.y_col['cls1']])
        self.df[self.y_col['cls1']] = self.cls1_en.transform(self.df[self.y_col['cls1']])
#         self.n_type = df[y_col['type']].nunique()
    
    def on_epoch_end(self):
        if self.shuffle:
            self.df = self.df.sample(frac=1).reset_index(drop=True)
    
    def __get_input(self, path, target_size):
    
#         xmin, ymin, w, h = bbox['x'], bbox['y'], bbox['width'], bbox['height']

        image = tf.keras.preprocessing.image.load_img(path)
        image = tf.keras.preprocessing.image.img_to_array(image)
        image = tf.image.resize(image, (target_size[0], target_size[1])).numpy()
        
#         image_arr = image_arr[ymin:ymin+h, xmin:xmin+w]
#         image_arr = tf.image.resize(image_arr,(target_size[0], target_size[1])).numpy() 
        
        return image
    
    def __get_output(self, label, num_classes):
        return tf.keras.utils.to_categorical(label, num_classes=num_classes)
    
    def __get_data(self, batches):
        # Generates data containing batch_size samples
        print(len(batches))
        print(type(batches))
        path_batch = batches[self.X_col['path']]
#         bbox_batch = batches[self.X_col['bbox']]
        
        cls1_batch = batches[self.y_col['cls1']]
#         type_batch = batches[self.y_col['type']]

        X_batch = np.asarray([self.__get_input(x, self.input_size) for x in path_batch])

        y1_batch = np.asarray([self.__get_output(y, self.n_cls) for y in cls1_batch])
#         y1_batch = np.asarray([self.__get_output(y, self.n_type) for y in type_batch])

        return X_batch, y1_batch
    
    def __getitem__(self, index):
        
        batches = self.df[index * self.batch_size:(index + 1) * self.batch_size]
        X, y = self.__get_data(batches)        
        return X, y
    
    def __len__(self):
        return self.n // self.batch_size


In [4]:
root_dir = '/home/lab38/images'
sample_csv = '딸기_정상_수정본.csv'
df = pd.read_csv(os.path.join(root_dir, sample_csv))
df.head()

,image,date,height,width,type,disease,crop,area,grow,risk,points
0,V006_77_0_00_04_01_13_0_b02_20201203_0000_S01_...,2020-12-03,4032,3024,0,0,4,1,13,0,"[{'xtl': 1003, 'ytl': 1167, 'xbr': 2255, 'ybr'..."
1,V006_77_0_00_04_01_13_0_b02_20201203_0001_S01_...,2020-12-03,4032,3024,0,0,4,1,13,0,"[{'xtl': 600, 'ytl': 1087, 'xbr': 1628, 'ybr':..."
2,V006_77_0_00_04_01_13_0_b02_20201203_0002_S01_...,2020-12-03,4032,3024,0,0,4,1,13,0,"[{'xtl': 1003, 'ytl': 1217, 'xbr': 2503, 'ybr'..."
3,V006_77_0_00_04_01_13_0_b02_20201203_0003_S01_...,2020-12-03,4032,3024,0,0,4,1,13,0,"[{'xtl': 662, 'ytl': 1252, 'xbr': 2129, 'ybr':..."
4,V006_77_0_00_04_01_13_0_b02_20201203_0004_S01_...,2020-12-03,4032,3024,0,0,4,1,13,0,"[{'xtl': 370, 'ytl': 884, 'xbr': 1994, 'ybr': ..."


In [5]:
sample_img_path = os.path.join(root_dir, '딸기_정상')
df['image'] = df['image'].map(lambda x : os.path.join(sample_img_path, x))

In [6]:
df.head()

,image,date,height,width,type,disease,crop,area,grow,risk,points
0,/home/lab38/images/딸기_정상/V006_77_0_00_04_01_13...,2020-12-03,4032,3024,0,0,4,1,13,0,"[{'xtl': 1003, 'ytl': 1167, 'xbr': 2255, 'ybr'..."
1,/home/lab38/images/딸기_정상/V006_77_0_00_04_01_13...,2020-12-03,4032,3024,0,0,4,1,13,0,"[{'xtl': 600, 'ytl': 1087, 'xbr': 1628, 'ybr':..."
2,/home/lab38/images/딸기_정상/V006_77_0_00_04_01_13...,2020-12-03,4032,3024,0,0,4,1,13,0,"[{'xtl': 1003, 'ytl': 1217, 'xbr': 2503, 'ybr'..."
3,/home/lab38/images/딸기_정상/V006_77_0_00_04_01_13...,2020-12-03,4032,3024,0,0,4,1,13,0,"[{'xtl': 662, 'ytl': 1252, 'xbr': 2129, 'ybr':..."
4,/home/lab38/images/딸기_정상/V006_77_0_00_04_01_13...,2020-12-03,4032,3024,0,0,4,1,13,0,"[{'xtl': 370, 'ytl': 884, 'xbr': 1994, 'ybr': ..."


In [7]:
BATCH_SIZE = 20
IMAGE_SIZE = 320

In [8]:
shuffled_df = df.sample(frac=1)
# Train : Validation = 0.8 : 0.2 Split
train_len = int(len(df)*0.8)

train_df = shuffled_df[:train_len]
valid_df = shuffled_df[train_len:]

train_img_path = train_df['image'].to_list()
train_label = train_df['grow'].to_list()

valid_img_path = valid_df['image'].to_list()
valid_label = valid_df['grow'].to_list()

In [9]:
traingen = CustomDataGen(train_df,
                        X_col = {'path':'image'},
                        y_col = {'cls1' : 'grow'},
                        batch_size = BATCH_SIZE,
                        input_size = (IMAGE_SIZE, IMAGE_SIZE, 3))
validgen = CustomDataGen(valid_df,
                        X_col = {'path':'image'},
                        y_col = {'cls1' : 'grow'},
                        batch_size = BATCH_SIZE,
                        input_size = (IMAGE_SIZE, IMAGE_SIZE, 3))

In [10]:
inputs = keras.Input(shape=(320, 320, 3), name="img")
x = layers.Conv2D(16, 3, activation="relu")(inputs)
x = layers.Conv2D(32, 3, activation="relu")(x)
x = layers.MaxPooling2D(3)(x)
x = layers.Flatten()(x)
outputs = layers.Dense(1)(x)

model = keras.Model(inputs, outputs, name="test")
model.summary()

2022-06-09 14:11:40.100537: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-06-09 14:11:40.101576: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-06-09 14:11:40.149176: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-09 14:11:40.149731: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:1e.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2022-06-09 14:11:40.149770: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-06-09 14:11:40.151562: I tensorflow/stream_executor/platform/default/dso_loade

Model: "test"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
img (InputLayer)             [(None, 320, 320, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 318, 318, 16)      448       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 316, 316, 32)      4640      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 105, 105, 32)      0         
_________________________________________________________________
flatten (Flatten)            (None, 352800)            0         
_________________________________________________________________
dense (Dense)                (None, 1)                 352801    
Total params: 357,889
Trainable params: 357,889
Non-trainable params: 0
________________________________________________________

In [11]:
model.compile(loss='categorical_crossentropy',
             optimizer=Adam(learning_rate=1e-4),
              metrics=['accuracy'])

In [ ]:
model.fit(traingen,
         validation_data=validgen,
         epochs=1,
         verbose=1,
         batch_size = BATCH_SIZE,
         steps_per_epoch = len(traingen))

20
<class 'pandas.core.frame.DataFrame'>


2022-06-09 14:11:49.677056: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-06-09 14:11:49.694966: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2499995000 Hz
2022-06-09 14:11:50.016745: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


20
<class 'pandas.core.frame.DataFrame'>
20
<class 'pandas.core.frame.DataFrame'>


2022-06-09 14:11:56.979570: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7


  1/787 [..............................] - ETA: 1:56:45 - loss: 1.1921e-07 - accuracy: 0.500020
<class 'pandas.core.frame.DataFrame'>
  2/787 [..............................] - ETA: 1:10:05 - loss: 1.1921e-07 - accuracy: 0.500020
<class 'pandas.core.frame.DataFrame'>
  3/787 [..............................] - ETA: 1:20:26 - loss: 1.1921e-07 - accuracy: 0.500020
<class 'pandas.core.frame.DataFrame'>
  4/787 [..............................] - ETA: 1:23:43 - loss: 1.1921e-07 - accuracy: 0.500020
<class 'pandas.core.frame.DataFrame'>
  5/787 [..............................] - ETA: 1:24:49 - loss: 1.1921e-07 - accuracy: 0.500020
<class 'pandas.core.frame.DataFrame'>
  6/787 [..............................] - ETA: 1:23:44 - loss: 1.1921e-07 - accuracy: 0.500020
<class 'pandas.core.frame.DataFrame'>
  7/787 [..............................] - ETA: 1:24:02 - loss: 1.1921e-07 - accuracy: 0.500020
<class 'pandas.core.frame.DataFrame'>
  8/787 [..............................] - ETA: 1:23:52 - loss

...

555/787 [====================>.........] - ETA: 24:59 - loss: 1.1921e-07 - accuracy: 0.500020
<class 'pandas.core.frame.DataFrame'>
556/787 [====================>.........] - ETA: 24:53 - loss: 1.1921e-07 - accuracy: 0.500020
<class 'pandas.core.frame.DataFrame'>
557/787 [====================>.........] - ETA: 24:46 - loss: 1.1921e-07 - accuracy: 0.500020
<class 'pandas.core.frame.DataFrame'>
558/787 [====================>.........] - ETA: 24:40 - loss: 1.1921e-07 - accuracy: 0.500020
<class 'pandas.core.frame.DataFrame'>
559/787 [====================>.........] - ETA: 24:33 - loss: 1.1921e-07 - accuracy: 0.500020
<class 'pandas.core.frame.DataFrame'>
560/787 [====================>.........] - ETA: 24:27 - loss: 1.1921e-07 - accuracy: 0.500020
<class 'pandas.core.frame.DataFrame'>
561/787 [====================>.........] - ETA: 24:20 - loss: 1.1921e-07 - accuracy: 0.500020
<class 'pandas.core.frame.DataFrame'>
562/787 [====================>.........] - ETA: 24:14 - loss: 1.1921e-07 - a

617/787 [======================>.......] - ETA: 18:17 - loss: 1.1921e-07 - accuracy: 0.500020
<class 'pandas.core.frame.DataFrame'>
618/787 [======================>.......] - ETA: 18:11 - loss: 1.1921e-07 - accuracy: 0.500020
<class 'pandas.core.frame.DataFrame'>
619/787 [======================>.......] - ETA: 18:04 - loss: 1.1921e-07 - accuracy: 0.500020
<class 'pandas.core.frame.DataFrame'>
620/787 [======================>.......] - ETA: 17:58 - loss: 1.1921e-07 - accuracy: 0.500020
<class 'pandas.core.frame.DataFrame'>
621/787 [======================>.......] - ETA: 17:51 - loss: 1.1921e-07 - accuracy: 0.500020
<class 'pandas.core.frame.DataFrame'>
622/787 [======================>.......] - ETA: 17:45 - loss: 1.1921e-07 - accuracy: 0.500020
<class 'pandas.core.frame.DataFrame'>
623/787 [======================>.......] - ETA: 17:39 - loss: 1.1921e-07 - accuracy: 0.500020
<class 'pandas.core.frame.DataFrame'>
624/787 [======================>.......] - ETA: 17:32 - loss: 1.1921e-07 - a

679/787 [========================>.....] - ETA: 11:36 - loss: 1.1921e-07 - accuracy: 0.500020
<class 'pandas.core.frame.DataFrame'>
680/787 [========================>.....] - ETA: 11:29 - loss: 1.1921e-07 - accuracy: 0.500020
<class 'pandas.core.frame.DataFrame'>
681/787 [========================>.....] - ETA: 11:23 - loss: 1.1921e-07 - accuracy: 0.500020
<class 'pandas.core.frame.DataFrame'>
682/787 [========================>.....] - ETA: 11:17 - loss: 1.1921e-07 - accuracy: 0.500020
<class 'pandas.core.frame.DataFrame'>
683/787 [=========================>....] - ETA: 11:10 - loss: 1.1921e-07 - accuracy: 0.500020
<class 'pandas.core.frame.DataFrame'>
684/787 [=========================>....] - ETA: 11:04 - loss: 1.1921e-07 - accuracy: 0.500020
<class 'pandas.core.frame.DataFrame'>
685/787 [=========================>....] - ETA: 10:57 - loss: 1.1921e-07 - accuracy: 0.500020
<class 'pandas.core.frame.DataFrame'>
686/787 [=========================>....] - ETA: 10:51 - loss: 1.1921e-07 - a

741/787 [===========================>..] - ETA: 4:57 - loss: 1.1921e-07 - accuracy: 0.500020
<class 'pandas.core.frame.DataFrame'>
742/787 [===========================>..] - ETA: 4:50 - loss: 1.1921e-07 - accuracy: 0.500020
<class 'pandas.core.frame.DataFrame'>
743/787 [===========================>..] - ETA: 4:44 - loss: 1.1921e-07 - accuracy: 0.500020
<class 'pandas.core.frame.DataFrame'>
744/787 [===========================>..] - ETA: 4:37 - loss: 1.1921e-07 - accuracy: 0.500020
<class 'pandas.core.frame.DataFrame'>
745/787 [===========================>..] - ETA: 4:31 - loss: 1.1921e-07 - accuracy: 0.500020
<class 'pandas.core.frame.DataFrame'>
746/787 [===========================>..] - ETA: 4:24 - loss: 1.1921e-07 - accuracy: 0.500020
<class 'pandas.core.frame.DataFrame'>
747/787 [===========================>..] - ETA: 4:18 - loss: 1.1921e-07 - accuracy: 0.500020
<class 'pandas.core.frame.DataFrame'>
748/787 [===========================>..] - ETA: 4:11 - loss: 1.1921e-07 - accuracy:

20
<class 'pandas.core.frame.DataFrame'>
20
<class 'pandas.core.frame.DataFrame'>
20
<class 'pandas.core.frame.DataFrame'>
20
<class 'pandas.core.frame.DataFrame'>
20
<class 'pandas.core.frame.DataFrame'>
20
<class 'pandas.core.frame.DataFrame'>
20
<class 'pandas.core.frame.DataFrame'>
20
<class 'pandas.core.frame.DataFrame'>
20
<class 'pandas.core.frame.DataFrame'>
20
<class 'pandas.core.frame.DataFrame'>
20
<class 'pandas.core.frame.DataFrame'>
20
<class 'pandas.core.frame.DataFrame'>
20
<class 'pandas.core.frame.DataFrame'>
20
<class 'pandas.core.frame.DataFrame'>
20
<class 'pandas.core.frame.DataFrame'>
20
<class 'pandas.core.frame.DataFrame'>
20
<class 'pandas.core.frame.DataFrame'>
20
<class 'pandas.core.frame.DataFrame'>
20
<class 'pandas.core.frame.DataFrame'>
20
<class 'pandas.core.frame.DataFrame'>
20
<class 'pandas.core.frame.DataFrame'>
20
<class 'pandas.core.frame.DataFrame'>
20
<class 'pandas.core.frame.DataFrame'>
20
<class 'pandas.core.frame.DataFrame'>
20
<class 'panda